In [2]:
!pip install chromadb-client sentence_transformers
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 523.5/523.5 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.7/224.7 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.9/59.9 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.0/107.0 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.5/52.5 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.5/130.5 kB 8.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
 

In [8]:
import chromadb
from sentence_transformers import SentenceTransformer
import requests
import json
import os
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [5]:
class EmbeddingModel:
    def __init__(self, model):
        self.model = model

    def __call__(self, input):
        embeddings = []
        for text in input:
            if ' -- ' not in text:
                raise ValueError("Input text does not contain expected '--' delimiter.")

            question = text.split(' -- ')[0]

            embedded = self.model.encode(question, show_progress_bar=False)
            embeddings.append(embedded.tolist())

        return embeddings

embedding_model = EmbeddingModel(model)


In [7]:
client = chromadb.HttpClient(host='51.250.103.4', port=8000)
collection = client.get_collection(name="tinkoff_collection", embedding_function=embedding_model)

In [196]:
# def get_promt_on_check(questions, answer):

#   prompt = {
#     "modelUri": r"gpt://b1gtg0ar2bip3ebkst80/yandexgpt/latest", # версия модели тут https://yandex.cloud/ru/docs/foundation-models/concepts/yandexgpt/models
#     "completionOptions": {
#       "stream": False,
#       "temperature": 0.0,
#       "maxTokens": "2000"
#     },
#     "messages": [
#       { # тут задается промпт модели
#         "role": "system",
#         "text": f"""Тебе нужно бинарно оценить, была ли взята информация из приведенных источником. Выведи только 1 если 90% было взято оттуда и 0 если нет
# """
# },
#       { # тут вопрос пользователя
#         "role": "user",
#         "text": f"Источники:  {questions} \n Информация: {answer}"
#       }
#     ]
#   }

#   # with open(r"prompt.json", "w", encoding="utf-8") as json_file: # указать путь и имя json файла
#   #   json.dump(prompt, json_file, ensure_ascii=False)

#   return prompt

In [228]:
# secret_key = r'AQVN1G0EJLLoN3h1C9cwwR09wMiW5ZaQULveQ1vo'
# def get_promt(question, context):



#   prompt = {
#     "modelUri": r"gpt://b1gtg0ar2bip3ebkst80/yandexgpt/latest", # версия модели тут https://yandex.cloud/ru/docs/foundation-models/concepts/yandexgpt/models
#     "completionOptions": {
#       "stream": False,
#       "temperature": 0.0,
#       "maxTokens": "2000"
#     },
#     "messages": [
#       { # тут задается промпт модели
#         "role": "system",
#         "text": """Тебе будут предоставленны вопросы, ответами и ссылками. Далее последует вопрос от пользователя приложения банка Tinkoff, о котором ты знаешь ничего и не можешь давать общие советы
#   - Тебе нужно выбрать один ответ с самым подходящим ответом под новый вопрос
#   - Ты не можешь отвечать своими словами
#   - Выбери только один самый близкий вопрос, если он чуть-чуть отличается по смыслу, можешь немного изменить вступление
#   - Обязательно укажи источник в конце по шаблону
#   - Если ты не можешь ответить на вопрос исходя из данных, то скажи: К сожалению, я не могу ответить на этот вопрос.

#   Пример ввода:
#   Как открыть ИП онлайн? -- Есть несколько способов: через Тинькофф, сайт налоговой или Госуслуги. С помощью сервиса Тинькофф: Заполните анкету на сайте — займет 10 минут. Из документов нужны только СНИЛС и фото паспорта — главного разворота и страницы с пропиской. Назначьте встречу с представителем банка, чтобы подписать документы. Мы проверим документы и отправим их в налоговую. В течение рабочего дня вам на эл. почту придет подтверждение от ФНС, что документы получены. Через 3-5 рабочих дней на эл. почту придут документы о регистрации ИП: лист записи ЕГРИП и уведомление о постановке на учет в налоговую. Через сайт налоговой. Зайдите на сайт налоговой, заполните заявление о государственной регистрации ИП, выберите электронный способ подачи документов. Госпошлину платить не нужно, но понадобится квалифицированная электронная подпись, КЭП. Зарегистрировать ИП в ФНС База знаний ФНС по регистрации ИП и ООО Сервисы ФНС для регистрации ИП и ООО Через Госуслуги. Зайдите на портал госуслуг, авторизуйтесь, загрузите документы и заявление. Госпошлину платить не нужно. Понадобится подтвержденная учетная запись на портале и квалифицированная электронная подпись, КЭП. Как получить КЭП Зарегистрировать ИП через Госуслуги Открыть ИП можно также офлайн — в МФЦ, налоговой или через нотариуса. Подробнее об этих способах — в статье «Как открыть ИП: налоговый режим, ОКВЭДы, регистрация». -- https://www.tinkoff.ru/business/help/registration/registration/register-ip/open/?card=q1
#   Как открыть ИП, если нет постоянной регистрации? -- Чтобы стать ИП, подойдет и временная регистрация. Процесс такой же, как и с постоянной. После открытия ИП по временной регистрации вы будете относиться к инспекции ФНС Федеральная налоговая служба по месту пребывания. -- https://www.tinkoff.ru/business/help/registration/registration/register-ip/opening-conditions/?card=q5
#   Можно ли открыть ИП онлайн?

#   Пример вывода:
#   Открыть ИП можно онлайн. Есть несколько способов: через Тинькофф, сайт налоговой или Госуслуги. С помощью сервиса Тинькофф: Заполните анкету на сайте — займет 10 минут. Из документов нужны только СНИЛС и фото паспорта — главного разворота и страницы с пропиской. Назначьте встречу с представителем банка, чтобы подписать документы. Мы проверим документы и отправим их в налоговую. В течение рабочего дня вам на эл. почту придет подтверждение от ФНС, что документы получены. Через 3-5 рабочих дней на эл. почту придут документы о регистрации ИП: лист записи ЕГРИП и уведомление о постановке на учет в налоговую. Через сайт налоговой. Зайдите на сайт налоговой, заполните заявление о государственной регистрации ИП, выберите электронный способ подачи документов. Госпошлину платить не нужно, но понадобится квалифицированная электронная подпись, КЭП. Зарегистрировать ИП в ФНС База знаний ФНС по регистрации ИП и ООО Сервисы ФНС для регистрации ИП и ООО Через Госуслуги. Зайдите на портал госуслуг, авторизуйтесь, загрузите документы и заявление. Госпошлину платить не нужно. Понадобится подтвержденная учетная запись на портале и квалифицированная электронная подпись, КЭП. Как получить КЭП Зарегистрировать ИП через Госуслуги Открыть ИП можно также офлайн — в МФЦ, налоговой или через нотариуса. Подробнее об этих способах — в статье «Как открыть ИП: налоговый режим, ОКВЭДы, регистрация».
#   Источник: https://www.tinkoff.ru/business/help/registration/registration/register-ip/open/?card=q1
# """
# },
#       { # тут вопрос пользователя
#         "role": "user",
#         "text": f"Вопросы из базы: {context} \n Вопрос пользователя: {question}"
#       }
#     ]
#   }

#   # with open(r"prompt.json", "w", encoding="utf-8") as json_file: # указать путь и имя json файла
#   #   json.dump(prompt, json_file, ensure_ascii=False)

#   return prompt


# def get_relevant_questions(question, model = model):
#     results = collection.query(
#     query_texts=[question],
#     n_results=3
#     )
#     print(results)
#     return results['documents'], results['distances']
#     #return results['documents']



# def get_responce(auth_headers, question):

#     url = r'https://llm.api.cloud.yandex.net/foundationModels/v1/completion' # всегда такой

#     # with open(r'prompt.json', 'r', encoding='utf-8') as f: # создаем json в кодировке utf-8 и считываем файл (заменить путь)
#     #     data = json.dumps(json.load(f))
#     files = get_relevant_questions(question)
#     context = ''
#     for file in files[0][0]:
#       context += file
#       context += '\n'

#     prompt = get_promt(question, context)
#     data = json.dumps(prompt, ensure_ascii=True, indent=4)

#     resp = requests.post(url, headers=auth_headers, data=data)

#     if resp.status_code != 200:
#         raise RuntimeError(
#             'Invalid response received: code: {}, message: {}'.format(
#                 {resp.status_code}, {resp.text}
#             )
#         )

#     res_text = json.loads(resp.text)['result']['alternatives'][0]['message']['text']

#     prompt2 = get_promt_on_check(context, res_text)
#     print(prompt2)
#     data2 = json.dumps(prompt2, ensure_ascii=True, indent=4)

#     resp2 = requests.post(url, headers=auth_headers, data=data2)


#     if resp2.status_code != 200:
#         raise RuntimeError(
#             'Invalid response received: code: {}, message: {}'.format(
#                 {resp2.status_code}, {resp2.text}
#             )
#         )

#     prediction = json.loads(resp2.text)['result']['alternatives'][0]['message']['text']
#     print(prediction)
#     if prediction == '1':
#       return res_text
#     else:
#       return "К сожалению, я не могу ответить на этот вопрос."


# headers = {
#     'Content-Type' : 'application/json', # всегда
#     'Authorization' : f'Api-Key {secret_key}' # тут можно менять: Api-Key это ключ который выдается один раз при создании ключа, можно заменить на Bearer {iam_token}
# }




In [200]:
# question = 'Какой есть тинькофф для юр лиц -- '
# results = get_responce(headers, question)
# print(results)
# #как отписаться от ютуб канала

{'ids': [['id422', 'id2682']], 'distances': [[0.332852303981781, 0.332852303981781]], 'embeddings': None, 'metadatas': [[None, None]], 'documents': [['Карточка --  -- https://www.tinkoff.ru/business/help/business-self-employment/payout/about/features/?card=q1', 'Карточка --  -- https://www.tinkoff.ru/business/help/business-self-employment/self-employed/work/start/?card=q1']], 'uris': None, 'data': None}
{'modelUri': 'gpt://b1gtg0ar2bip3ebkst80/yandexgpt/latest', 'completionOptions': {'stream': False, 'temperature': 0.0, 'maxTokens': '2000'}, 'messages': [{'role': 'system', 'text': 'Тебе нужно бинарно оценить, была ли взята информация из приведенных источником. Выведи только 1 если 90% было взято оттуда и 0 если нет\n'}, {'role': 'user', 'text': 'Источники:  Карточка --  -- https://www.tinkoff.ru/business/help/business-self-employment/payout/about/features/?card=q1\nКарточка --  -- https://www.tinkoff.ru/business/help/business-self-employment/self-employed/work/start/?card=q1\n \n Инфор

In [133]:
# results.text

'{"result":{"alternatives":[{"message":{"role":"assistant","text":"Чтобы отписаться от канала на YouTube, необходимо выполнить следующие шаги:\\n\\n1. Открыть YouTube на компьютере или мобильном устройстве.\\n2. Перейти в раздел «Подписки».\\n3. Найти нужный канал и нажать на кнопку «Отписаться».\\n4. Подтвердить действие.\\n\\nТакже можно отписаться от каналов на YouTube через мобильное приложение или веб-версию сервиса. Для этого нужно найти интересующий канал в списке подписок и нажать кнопку «Отписаться» или «Unsubscribe»."},"status":"ALTERNATIVE_STATUS_FINAL"}],"usage":{"inputTextTokens":"1055","completionTokens":"99","totalTokens":"1154"},"modelVersion":"07.03.2024"}}\n'

In [38]:
# results.text

'{"result":{"alternatives":[{"message":{"role":"assistant","text":"Открыть ИП можно онлайн. Есть несколько способов: через Тинькофф, сайт налоговой или Госуслуги. С помощью сервиса Тинькофф: Заполните анкету на сайте — займёт 10 минут. Из документов нужны только СНИЛС и фото паспорта — главного разворота и страницы с пропиской. Назначьте встречу с представителем банка, чтобы подписать документы. Мы проверим документы и отправим их в налоговую. В течение рабочего дня вам на эл. почту придёт подтверждение от ФНС, что документы получены. Через 3–5 рабочих дней на эл. почту придут документы о регистрации ИП: лист записи ЕГРИП и уведомление о постановке на учёт в налоговую.\\n\\nЧерез сайт налоговой. Зайдите на сайт налоговой, заполните заявление о государственной регистрации ИП, выберите электронный способ подачи документов. Госпошлину платить не нужно, но понадобится квалифицированная электронная подпись, КЭП. Зарегистрировать ИП в ФНС База знаний ФНС по регистрации ИП и ООО Сервисы ФНС д

In [234]:
from openai import OpenAI
import re
client = OpenAI(api_key='sk-proj-SyXBjT4PPB98njKb7GYFT3BlbkFJg3ItqCdr7WzBe7zHTr6b')

def get_relevant_questions(question, model = model):
    results = collection.query(
    query_texts=[question],
    n_results=3
    )
    print(results)
    return results['documents'], results['distances']

def generate_questions(data, model_name="gpt-3.5-turbo"):
    completion = client.chat.completions.create(
        model=model_name,
        temperature = 0.0,
        messages=[
            {"role": "system",
             "content": f"""Тебе будут предоставленны вопросы, с ответами и ссылками. Далее последует вопрос от пользователя приложения банка Tinkoff
  - Ты не можешь отвечать своими словами
  - Выбери только один самый близкий вопрос, если он чуть-чуть отличается по смыслу, можешь немного изменить вступление
  - Обязательно укажи источник в конце по шаблону
  - Если ты не можешь ответить на вопрос исходя из данных, то скажи: К сожалению, я не могу ответить на этот вопрос.

  Пример ввода:
  Как открыть ИП онлайн? -- Есть несколько способов: через Тинькофф, сайт налоговой или Госуслуги. С помощью сервиса Тинькофф: Заполните анкету на сайте — займет 10 минут. Из документов нужны только СНИЛС и фото паспорта — главного разворота и страницы с пропиской. Назначьте встречу с представителем банка, чтобы подписать документы. Мы проверим документы и отправим их в налоговую. В течение рабочего дня вам на эл. почту придет подтверждение от ФНС, что документы получены. Через 3-5 рабочих дней на эл. почту придут документы о регистрации ИП: лист записи ЕГРИП и уведомление о постановке на учет в налоговую. Через сайт налоговой. Зайдите на сайт налоговой, заполните заявление о государственной регистрации ИП, выберите электронный способ подачи документов. Госпошлину платить не нужно, но понадобится квалифицированная электронная подпись, КЭП. Зарегистрировать ИП в ФНС База знаний ФНС по регистрации ИП и ООО Сервисы ФНС для регистрации ИП и ООО Через Госуслуги. Зайдите на портал госуслуг, авторизуйтесь, загрузите документы и заявление. Госпошлину платить не нужно. Понадобится подтвержденная учетная запись на портале и квалифицированная электронная подпись, КЭП. Как получить КЭП Зарегистрировать ИП через Госуслуги Открыть ИП можно также офлайн — в МФЦ, налоговой или через нотариуса. Подробнее об этих способах — в статье «Как открыть ИП: налоговый режим, ОКВЭДы, регистрация». -- https://www.tinkoff.ru/business/help/registration/registration/register-ip/open/?card=q1
  Как открыть ИП, если нет постоянной регистрации? -- Чтобы стать ИП, подойдет и временная регистрация. Процесс такой же, как и с постоянной. После открытия ИП по временной регистрации вы будете относиться к инспекции ФНС Федеральная налоговая служба по месту пребывания. -- https://www.tinkoff.ru/business/help/registration/registration/register-ip/opening-conditions/?card=q5
  Можно ли открыть ИП онлайн?

  Пример вывода:
  Открыть ИП можно онлайн. Есть несколько способов: через Тинькофф, сайт налоговой или Госуслуги. С помощью сервиса Тинькофф: Заполните анкету на сайте — займет 10 минут. Из документов нужны только СНИЛС и фото паспорта — главного разворота и страницы с пропиской. Назначьте встречу с представителем банка, чтобы подписать документы. Мы проверим документы и отправим их в налоговую. В течение рабочего дня вам на эл. почту придет подтверждение от ФНС, что документы получены. Через 3-5 рабочих дней на эл. почту придут документы о регистрации ИП: лист записи ЕГРИП и уведомление о постановке на учет в налоговую. Через сайт налоговой. Зайдите на сайт налоговой, заполните заявление о государственной регистрации ИП, выберите электронный способ подачи документов. Госпошлину платить не нужно, но понадобится квалифицированная электронная подпись, КЭП. Зарегистрировать ИП в ФНС База знаний ФНС по регистрации ИП и ООО Сервисы ФНС для регистрации ИП и ООО Через Госуслуги. Зайдите на портал госуслуг, авторизуйтесь, загрузите документы и заявление. Госпошлину платить не нужно. Понадобится подтвержденная учетная запись на портале и квалифицированная электронная подпись, КЭП. Как получить КЭП Зарегистрировать ИП через Госуслуги Открыть ИП можно также офлайн — в МФЦ, налоговой или через нотариуса. Подробнее об этих способах — в статье «Как открыть ИП: налоговый режим, ОКВЭДы, регистрация».
  Источник: https://www.tinkoff.ru/business/help/registration/registration/register-ip/open/?card=q1
    {data}"""},
        ]
    )
    return completion.choices[0].message

def check_answer(answer, data, model_name="gpt-3.5-turbo"):
    completion = client.chat.completions.create(
        model=model_name,
        temperature = 0.0,
        messages=[
            {"role": "system",
             "content": f""" Ты должен бинарно проверить ответ на галюцинации, то есть была ли информация взята с этих источников или нет. Выведи только 1 если да и 0 если нет.
             Также выведи 0 если ответ является коротким или не по теме банков и финансов
             Информация: {answer} \n
             Источники: {data}
"""},
        ]
    )
    return completion.choices[0].message

def generate_answer(question):
    top_close, prob = get_relevant_questions(question)
    print(prob)
    context = ''
    for file in top_close[0]:
      context += file
      context += '\n'
    context += question
    if  prob[0][0] > 0.5:
        return "К сожалению, я не могу ответить на этот вопрос. "
    else:
        print(context)
        text = generate_questions(context).content

        if check_answer(text, context).content == '1':
          return text
        else:
          return " К сожалению, я не могу ответить на этот вопрос."


url_pattern = r'https?://[^\s,]+'

test_question = 'Как проследить за бухгалтером'
def get_answer(question):
  test_question = question + ' -- '
  res = generate_answer(test_question)
  urls = re.findall(url_pattern, res)
  if len(urls) == 0:
    urls.append('')

  return res, urls[0]

print(get_answer('Как проследить за бухгалтером'))



{'ids': [['id3470', 'id6014', 'id2777']], 'distances': [[0.13781768083572388, 0.13781768083572388, 0.14910399913787842]], 'embeddings': None, 'metadatas': [[None, None, None]], 'documents': [['Каким образом можно связаться с бухгалтером? -- Общаться с бухгалтером можно через чат в личном кабинете и бесплатное приложение для iOS и Android. Или ему можно позвонить — для этого свяжитесь с персональным менеджером. -- https://www.tinkoff.ru/business/help/reports/accounting-service/about/personal-accounting/?card=write-accountant', 'Каким образом можно связаться с бухгалтером? -- Общаться с бухгалтером можно через чат в личном кабинете и бесплатном приложении для Android. Или можно позвонить ему по номеру 8 800 511-98-42 в будни с 10:00 до 19:00 мск. Как скачать приложение Тинькофф Бизнеса -- https://www.tinkoff.ru/business/help/reports/tinkoff-accounting/accounting-service/personal-accounting/?card=q9', 'Как связаться с бухгалтером? -- Общаться с бухгалтером можно через чат в личном кабинет

In [ ]:
"""
        """